# Populate our RDF database

### Imports and data loading

In [264]:
# required libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from IPython.display import clear_output

#!pip install rdflib
from rdflib import Graph, Literal, RDF, URIRef, Namespace, term
from rdflib.namespace import FOAF, XSD

#season parameters
y = 2019
shortSeas = "1920"
fullSeas = str(y)+"-"+str(y+1)

# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())
statsUrl = path + "/inDepthSoccerStats/"+fullSeas+".csv"
statsFBrefUrl = path + "/inDepthSoccerStats/transfermarkt_fbref_20"+shortSeas+".csv"
playersUrl = path + '/inDepthSoccerStats/players.csv'
teamsUrl = path + '/inDepthSoccerStats/clubs.csv'
appUrl = path + '/inDepthSoccerStats/appearances.csv'
gamesUrl = path + '/inDepthSoccerStats/games.csv'

# country codes
countriesURL = path + '/inDepthSoccerStats/wikipedia-iso-country-codes.csv'

# saving folder
savePath =  path + '/rdf/'

In [265]:
# Load the CSV files in memory
stats = pd.read_csv(statsUrl, sep=',', index_col='indCol')
#these dataframes store data from Transfermarkt
players = pd.read_csv(playersUrl, sep=',', index_col='player_id')
teams = pd.read_csv(teamsUrl, sep=',', index_col='club_id')
app = pd.read_csv(appUrl, sep=',', index_col='appearance_id')
games = pd.read_csv(gamesUrl, sep=',', index_col='game_id')
#FBref file used for completing some missing data. Available only for 3 seasons.
if(y >= 2017):
    statsFBref = pd.read_csv(statsFBrefUrl, sep=';', index_col='Column1', dtype={"Attendance": "string"})

#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
countries = pd.read_csv(countriesURL, sep=',', index_col='English short name lower case', keep_default_na=False, na_values=['_'])

#players.info()

### Parsing and matching utilities

In [60]:
import datetime
#!pip install unidecode
#!pip install googlesearch-python
from unidecode import unidecode
from itertools import permutations
from difflib import SequenceMatcher
from googlesearch import search

#string parsing

def hyphenize(s):
    return unidecode(s.lower().replace(" ","-").replace("&#039;","'"))

def cleanChars(item):
    return item.str.replace("ć", "c").str.replace("ğ","g").str.replace("İ","i").str.replace("-scaron-","s")

def cleanString(s):
    return unidecode(s.replace("ć", "c").replace("ğ","g").replace("İ","i"))
    

#permutations strategy

def genSeqByLength(perm, length):
    newName = ""
    for j in range(0, length - 1):
        newName = newName + perm[j] + "-"
    newName = newName + perm[length - 1]
    return newName

def match_seq(splitS1, lis):
    resultList = []
    #iterate on all possible permutations
    length = -1
    if(len(lis) == 1):
        length = len(lis[0].split("-"))
        uniqueItem = lis[0]
    for perm in permutations(splitS1):
        if(length != -1):
            newName = genSeqByLength(perm, length)
            if(uniqueItem == newName):
                return [uniqueItem]
        #consider all lengths from 2 to n
        else:
            for i in range(2, len(splitS1) + 1):
                newName = genSeqByLength(perm, i)
                for item in lis:
                    if(item == newName):
                        resultList.append(newName)
    return resultList

#multiple matches resolution

def getAppsByID(ID, apps):
    return np.size(apps[apps['player_id'] == ID], 0);

#I: number of games in the season, candidate players, list describing single appearances
#O: player from players file, or empty Series
def solve_with_apps_approx(statsGames, somePlayers, appsCol, tol=5):
    minDiff = 50
    minInd = 0
    for ind in somePlayers.index:
        if(abs(statsGames - getAppsByID(ind, appsCol)) < minDiff):
            minDiff = abs(statsGames - getAppsByID(ind, appsCol))
            minInd = ind
    if(minDiff <= tol):
        player = somePlayers[somePlayers.index == minInd].iloc[0] 
        return player, minInd
    else:
        return pd.Series([]), -1

## Matching

### Matching teams from different sources
This section is intended to create a dictionary to match team names in stats files to their Transfermarkt ID. This operation can be performed just once, as the dictionary can be dumped to a file.

In [210]:
%%time

statsTeamsSet = set()
for n in range(2014, 2020):
    tmpstatsUrl = path + "/inDepthSoccerStats/"+str(n)+"-"+str(n+1)+".csv"
    tmpstats = pd.read_csv(tmpstatsUrl, sep=',', index_col='indCol')
    for ind, row in tmpstats.iterrows():
        currTeams = row['teams_played_for'].split(",")
        statsTeamsSet.update(currTeams)

statsTeams = list(statsTeamsSet)
teamIDDict = dict()
i = 0
for statsTeam in statsTeams:
    i += 1
    maxS = 0
    maxId = 0
    for tind, trow in teams.iterrows():
        sm = SequenceMatcher(None, statsTeam, trow['name'])
        sim = sm.ratio()
        if(sim > maxS):
            maxS = sim
            maxId = tind

    if(maxS < 0.95):
        splitURL = next(search(statsTeam+" transfermarkt startseite verein", num_results=1)).split("/")
        #some teams contain numbers in their name, so we need to take only the suffix of the URL
        trID = splitURL[len(splitURL) - 1]
        if(trID != ""):
            if(len(teams[teams.index == int(trID)]) == 1):
                maxId = int(trID)
                print("{:3d}".format(i)+" out of "+"{:3d}".format(len(statsTeams))+" GOOGLE: "+statsTeam+" --> "+teams.at[maxId, 'name'])
            else:
                print("Invalid ID extracted from "+URL)
        else:
            print("No ID in URL "+URL)

    teamIDDict[statsTeam] = maxId

  1 out of 143 GOOGLE: Athletic Club --> Athletic Bilbao
  2 out of 143 GOOGLE: Fulham --> Fulham FC
  3 out of 143 GOOGLE: Lille --> LOSC Lille
  4 out of 143 GOOGLE: Torino --> Torino FC
  5 out of 143 GOOGLE: Mallorca --> RCD Mallorca
  6 out of 143 GOOGLE: Roma --> AS Roma
  8 out of 143 GOOGLE: Levante --> Levante UD
  9 out of 143 GOOGLE: Atletico Madrid --> Atlético de Madrid
 10 out of 143 GOOGLE: Brescia --> Brescia Calcio
 11 out of 143 GOOGLE: Celta Vigo --> Celta de Vigo
 12 out of 143 GOOGLE: Barcelona --> FC Barcelona
 13 out of 143 GOOGLE: Monaco --> AS Monaco
 15 out of 143 GOOGLE: Bayer Leverkusen --> Bayer 04 Leverkusen
 16 out of 143 GOOGLE: SPAL 2013 --> SPAL
 17 out of 143 GOOGLE: Hoffenheim --> TSG 1899 Hoffenheim
 19 out of 143 GOOGLE: Hull --> Hull City
 20 out of 143 GOOGLE: Deportivo La Coruna --> Deportivo de La Coruña
 21 out of 143 GOOGLE: Middlesbrough --> Middlesbrough FC
 23 out of 143 GOOGLE: Verona --> Hellas Verona
 24 out of 143 GOOGLE: Rennes --> St

We dump the dictionary to a file in the working directory.

In [211]:
import pickle
with open('teamdict.pkl', 'wb') as f:
    pickle.dump(teamIDDict, f)

3


### Matching players from different sources

We load the dictionary with team IDs which is saved in the working directory.

In [257]:
import pickle
with open('teamdict.pkl', 'rb') as f:
    teamIDDict = pickle.load(f)    

We add 3 new columns to the dataframe to insert also Transfermarkt IDs of the player and the teams he has played for.

In [266]:
stats['trID'] = [0] * len(stats)
stats['team0ID'] = [0] * len(stats)
stats['team1ID'] = [0] * len(stats)

In [267]:
%%time

#select only months and years from appearance dates
appMonth = app['date'].str.split("/").str[1]
appYear = app['date'].str.split("/").str[2]
#select appearances from this season
isThisSeas = ((appYear == str(y)) & (appMonth >= "08")) | ((appYear == str(y+1)) & (appMonth <= "06"))
appThisSeas = app[isThisSeas]
playerCodes = cleanChars(players['player_code'])

#iterate on stats file
statsRows = np.size(stats, 0);
toComplRows = len(stats[stats['trID'] == 0])
exact_matches = no_matches = resolved_google = resolved_permS = resolved_permP = resolved_max_sim = resolved_pres = i = 0
for index, row in stats.iterrows():
    if(row['trID'] == 0):
        i += 1
        mode = "NONE"
        player = pd.Series([])
        statsName = hyphenize(row['player_name']).replace("'","")
        
        matchedPlayers = players[playerCodes == statsName]
    
        #multiple rows with same name in stats mapped to a single player are ok (he has changed team during the season)
        if(np.size(matchedPlayers, 0) > 0):
            mode = "MATCH"
           
        if(mode == "NONE"):
            #split name in stats and use permutations strategy
            splitStatsName = statsName.split("-")
            if(len(splitStatsName) >= 2):
                matchedCodes = match_seq(splitStatsName, playerCodes)
                if(len(matchedCodes) > 0):
                    matchedPlayers = players[playerCodes.isin(matchedCodes)]
                    mode = "PERM1"
    
        if(mode == "NONE"):
            maxSim = 0
            maxC = ""
            for c in playerCodes:
                sm = SequenceMatcher(None, statsName, c)
                #do not proceed if the upper bound is too small
                if(sm.real_quick_ratio() >= 0.5):
                    #remember: similarity is not commutative
                    simm = sm.ratio()                  
                    #if sim is big enough, try permutation strategy with name from players file
                    if(simm >= 0.6):
                        splitC = c.split("-")
                        if(len(splitC) >= len(splitStatsName) and len(splitStatsName) >= 2):
                            matchedCodes = match_seq(splitC, [statsName])
                            if(len(matchedCodes) > 0):
                                newMatchedPlayers = players[players['player_code'] == c]
                                matchedPlayers = pd.concat([matchedPlayers, newMatchedPlayers])
                    if(simm > maxSim):
                        maxSim = simm
                        maxC = c
    
            if(maxSim >= 0.95):
                matchedPlayers = players[playerCodes == maxC]
                mode = "MAXSIM"
            elif(np.size(matchedPlayers, 0) > 0):
                mode = "PERM2"
    
        #managing results of any method
        matches = np.size(matchedPlayers, 0)
        if(matches == 1):
            player = matchedPlayers.iloc[0]
            trID = matchedPlayers.index[0]
        if(matches > 1):
            player, trID = solve_with_apps_approx(row['games'], matchedPlayers, appThisSeas, 3)
            if(trID == -1):
                mode = "NONE"
            else:
                mode = "PRES"
        if(mode == "NONE"):
            splitURL = next(search(row['player_name']+" "+row['teams_played_for']+" transfermarkt profil spieler", num_results=1)).split("/")
            trID = splitURL[len(splitURL) - 1]
            if(trID != ""):
                urlPlayers = players[players.index == int(trID)]
                if(len(urlPlayers) != 0):
                    mode = "GOOGLE"
                    player = urlPlayers.iloc[0]
                else:
                    print("!!! Invalid ID "+trID+" extracted from "+str(splitURL))
                    mode = "NONE"
            else:
                print("!!! No ID in URL "+str(splitURL))
                mode = "NONE"
        
        
        if(mode == "NONE"):
            no_matches += 1
        elif(mode == "PERM1"):
            resolved_permS += 1
        elif(mode == "PERM2"):
            resolved_permP += 1
        elif(mode == "GOOGLE"):
            resolved_google += 1
        elif(mode == "MATCH"):
            exact_matches += 1
        elif(mode == "MAXSIM"):
            resolved_max_sim += 1
        elif(mode == "PRES"):
            resolved_pres += 1
    
        if(mode == "NONE"):
            print("{:4d}".format(i)+" out of "+str(toComplRows)+" NONE  : "+statsName+", "+row['teams_played_for']+", matches: "+str(matches))
        elif(mode != "MATCH"):
            print("{:4d}".format(i)+" out of "+str(toComplRows)+" "+mode.ljust(6)+": "+statsName+" --> "+player['player_code'])
    
        if(mode != "NONE"):
            stats.at[index, 'trID'] = int(trID)
            teamList = row['teams_played_for'].split(",")
            stats.at[index, 'team0ID'] = int(teamIDDict[teamList[0]])
            if(len(teamList) == 2):
                stats.at[index, 'team1ID'] = int(teamIDDict[teamList[1]])

  18 out of 2661 PERM1 : son-heung-min --> heung-min-son
  28 out of 2661 GOOGLE: willian --> willian
  30 out of 2661 GOOGLE: callum-wilson --> callum-wilson
  43 out of 2661 GOOGLE: oliver-mcburnie --> oli-mcburnie
  50 out of 2661 GOOGLE: trezeguet --> mahmoud-trezeguet
  51 out of 2661 GOOGLE: david-silva --> david-silva
  64 out of 2661 GOOGLE: wesley --> wesley-moraes
  73 out of 2661 GOOGLE: adama-traore --> adama-traore
  75 out of 2661 GOOGLE: jorginho --> jorginho
  95 out of 2661 GOOGLE: rodri --> rodri
 119 out of 2661 GOOGLE: sokratis --> sokratis-papastathopoulos
 150 out of 2661 GOOGLE: jonny --> jonny-otto
 152 out of 2661 GOOGLE: fabinho --> fabinho
 158 out of 2661 PERM1 : tanguy-ndombele-alvaro --> tanguy-ndombele
 172 out of 2661 GOOGLE: matthew-longstaff --> matty-longstaff
 193 out of 2661 PRES  : pedro --> pedro
 207 out of 2661 GOOGLE: tom-davies --> tom-davies
 225 out of 2661 GOOGLE: joao-moutinho --> joao-moutinho
 256 out of 2661 PERM1 : ezri-konsa-ngoyo -->

In [268]:
#print statistics
print("   --- STATISTICS ---")
tot_matches = exact_matches + + resolved_google + resolved_permS + resolved_permP + resolved_max_sim + resolved_pres
print("Total matches:                            "+"{:5d}".format(tot_matches)+" -- percentage: " + "{:.2f}%".format(tot_matches*100/statsRows))
print("  ---> exact matches:                     "+"{:5d}".format(exact_matches)+" -- percentage: " + "{:.2f}%".format(exact_matches*100/statsRows)) 
print("  ---> resolved permutating statsName:    "+"{:5d}".format(resolved_permS)+" -- percentage: " + "{:.2f}%".format(resolved_permS*100/statsRows))
print("  ---> resolved permutating player code:  "+"{:5d}".format(resolved_permP)+" -- percentage: " + "{:.2f}%".format(resolved_permP*100/statsRows))
print("  ---> resolved with max sim.:            "+"{:5d}".format(resolved_max_sim)+" -- percentage: " + "{:.2f}%".format(resolved_max_sim*100/statsRows))
print("  ---> resolved with apps:                "+"{:5d}".format(resolved_pres)+" -- percentage: " + "{:.2f}%".format(resolved_pres*100/statsRows))
print("  ---> resolved with google:              "+"{:5d}".format(resolved_google)+" -- percentage: " + "{:.2f}%".format(resolved_google*100/statsRows))
print("No matches:                               "+"{:5d}".format(no_matches)+" -- percentage: " + "{:.2f}%".format(no_matches*100/statsRows))
print("  ---> zero matches found:                "+"{:5d}".format(no_matches)+" -- percentage: " + "{:.2f}%".format((no_matches)*100/statsRows))

   --- STATISTICS ---
Total matches:                             2661 -- percentage: 100.00%
  ---> exact matches:                      2376 -- percentage: 89.29%
  ---> resolved permutating statsName:       40 -- percentage: 1.50%
  ---> resolved permutating player code:      7 -- percentage: 0.26%
  ---> resolved with max sim.:               15 -- percentage: 0.56%
  ---> resolved with apps:                   55 -- percentage: 2.07%
  ---> resolved with google:                168 -- percentage: 6.31%
No matches:                                   0 -- percentage: 0.00%
  ---> zero matches found:                    0 -- percentage: 0.00%


Now the dataframe contains TM IDs, which will be used like foreign keys for matching purposes. Therefore, we save the dataframe.

In [269]:
#save the dataframe
stats.to_csv("stats"+shortSeas+"_IDs.csv")

#### Completing and correcting statistics in corner cases
We need to manage the fact some rows contain total information about a player switching team in the same league during the season. <br>
We can use information in the FBref files to complete our data; we have observed that, in this situation, it contains correct information only for the row of the two which has lower index: some statistics in the second row can be therefore corrected by subtracting the ones in the first row from the total ones. <br>

Observe that we load stats to make this cell idempotent.

In [270]:
#we will use copies of row of the df, so we silence the warnings that would arise
pd.options.mode.chained_assignment = None

statsFBref = pd.read_csv(statsFBrefUrl, sep=';', index_col='Column1', dtype={"Attendance": "string"})
statsFBref['player'] = [unidecode(name) for name in statsFBref['player']]

path = str(Path(os.path.abspath(os.getcwd())))
statsUrl = path + '/stats'+shortSeas+'_IDs.csv'
stats = pd.read_csv(statsUrl, sep=',', index_col='indCol')

m = nm = 0
newRows = []
for ind, row in stats.iterrows():
    if(row['team1ID'] != 0):
        player = players.loc[row['trID']]
        fbmatch = statsFBref[statsFBref['player'] == unidecode(player['name'])]
        if(len(fbmatch) == 2):
            m += 1
            team0ID, team1ID = row['team0ID'], row['team1ID']
            team0 = teams.loc[team0ID]['name']

            #select the index pointing to wrong row
            if(fbmatch.index[0] < fbmatch.index[1]):
                corrInd, wrongInd = fbmatch.index
            else:
                wrongInd, corrInd = fbmatch.index

            sm0 = SequenceMatcher(None, team0, fbmatch.loc[corrInd]['squad'])
            sm1 = SequenceMatcher(None, team0, fbmatch.loc[wrongInd]['squad'])

            #if the team corresponding to first ID matches with fbref row with index wrongInd, swap team IDs
            if(sm0.ratio() < sm1.ratio()):
                team0ID, team1ID = row['team1ID'], row['team0ID']

            corrRow = stats.loc[ind]
            wrongRow = stats.loc[ind]
            
            corrRow.at['teams_played_for'] = teams.loc[team0ID]['name']
            corrRow.at['team0ID'] = team0ID
            corrRow.at['team1ID'] = 0
            corrRow.at['games'] = statsFBref.at[corrInd, 'games']
            corrRow.at['goals'] = statsFBref.at[corrInd, 'goals']
            corrRow.at['minutes_played'] = statsFBref.at[corrInd, 'minutes']
            corrRow.at['npg'] = statsFBref.at[corrInd, 'goals'] - statsFBref.at[corrInd, 'pens_made']
            corrRow.at['assists'] = statsFBref.at[corrInd, 'assists']
            corrRow.at['xG'] = statsFBref.at[corrInd, 'xg']
            corrRow.at['xG90'] = corrRow['xG'] / corrRow['minutes_played'] * 90
            corrRow.at['npxG'] = statsFBref.at[corrInd, 'npxg']
            corrRow.at['npxG90'] = corrRow['npxG'] / corrRow['minutes_played'] * 90
            corrRow.at['xA'] = statsFBref.at[corrInd, 'xa']
            corrRow.at['xA90'] = corrRow['xA'] / corrRow['minutes_played'] * 90
            corrRow.at['yellow_cards'] = statsFBref.at[corrInd, 'cards_yellow']
            corrRow.at['red_cards'] = statsFBref.at[corrInd, 'cards_red']
            corrRow.at['shots'] = statsFBref.at[corrInd, 'shots_total']

            wrongRow.at['teams_played_for'] = teams.loc[team1ID]['name']
            wrongRow.at['team0ID'] = team1ID
            wrongRow.at['team1ID'] = 0
            wrongRow.at['games'] = row['games'] - corrRow['games']
            wrongRow.at['goals'] = row['goals'] - corrRow['goals']
            wrongRow.at['minutes_played'] = row['minutes_played'] - corrRow['minutes_played']
            wrongRow.at['npg'] = row['npg'] - corrRow['npg']
            wrongRow.at['assists'] = row['assists'] - corrRow['assists']
            #xG could result < 0 for different roundings or slightly different calculations
            #therefore, in such cases, setting it to 0 could be reasonable
            wrongRow.at['xG'] = max(row['xG'] - corrRow['xG'], 0)
            wrongRow.at['xG90'] = wrongRow['xG'] / wrongRow['minutes_played'] * 90
            wrongRow.at['npxG'] = max(row['npxG'] - corrRow['npxG'], 0)
            wrongRow.at['npxG90'] = wrongRow['npxG'] / wrongRow['minutes_played'] * 90
            wrongRow.at['xA'] = max(row['xA'] - corrRow['xA'], 0)
            wrongRow.at['xA90'] = wrongRow['xA'] / wrongRow['minutes_played'] * 90
            wrongRow.at['yellow_cards'] = row['yellow_cards'] - corrRow['yellow_cards']
            #shots could result negative for the same reason as above
            wrongRow.at['shots'] = max(row['shots'] - corrRow['shots'], 0)

            #we lack of data for these columns, so we mark this lack for later
            corrRow.at['xGBuildup'] = corrRow['xGChain'] = wrongRow.at['xGBuildup'] = wrongRow['xGChain'] = -1
            corrRow.at['key_passes'] = wrongRow.at['key_passes'] = -1
            
            newRows.append(corrRow.values)
            newRows.append(wrongRow.values)

        else:
            nm += 1
            print(row['player_name'])
            
print("Could match: "+str(m)+" could not: "+str(nm))
#create the new dataframe to be appended to the old one
startInd = stats.index[len(stats) - 1] + 1
newStats = pd.DataFrame(newRows, columns=stats.columns, index=range(startInd,startInd+len(newRows)))
stats = pd.concat([stats, newStats])

Cédric Soares
José Arnáiz
Carles Aleñá
Alejandro Pozo
Kévin Rodrigues
Bryan Gil Salvatierra
Bruno
Raúl García
Eduard Löwen
Antonin Barak
Rômulo
Nehuén Paz
Igor Julio
Jeff Reine-Adelaide
Adama Traoré
Could match: 56 could not: 15


In [271]:
#save the dataframe
stats.to_csv("stats"+shortSeas+"_IDs.csv")

## Namespaces and prefixes

This snippet is repeated below to save serializations in different files.

In [272]:
# Construct the soccer stats ontology namespace
#CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
#g.bind("countries", CNS)
g.bind("dcsso", DCSSO)

## Graph population

#### Countries

In [ ]:
%%time

for ind, row in countries.iterrows():
    country = URIRef(DCSSO[row['Alpha-2 code']])
    g.add((country, RDF.type, DCSSO.Country))
    g.add((country, FOAF.name, Literal(cleanString(ind), datatype=XSD.string)))

#### Leagues
Leagues are added manually because we need to store only five using very limited information from the file.

In [ ]:
%%time

SerieA = URIRef(DCSSO["IT1"])
g.add((SerieA, RDF.type, DCSSO.League))
g.add((SerieA, FOAF['name'], Literal("Serie A", datatype=XSD.string)))
g.add((SerieA, DCSSO['hasCountry'], URIRef(DCSSO["IT"])))

Ligue1 = URIRef(DCSSO["FR1"])
g.add((Ligue1, RDF.type, DCSSO.League))
g.add((Ligue1, FOAF['name'], Literal("Ligue 1", datatype=XSD.string)))
g.add((Ligue1, DCSSO['hasCountry'], URIRef(DCSSO["FR"])))

LaLiga = URIRef(DCSSO["ES1"])
g.add((LaLiga, RDF.type, DCSSO.League))
g.add((LaLiga, FOAF['name'], Literal("LaLiga", datatype=XSD.string)))
g.add((LaLiga, DCSSO['hasCountry'], URIRef(DCSSO["ES"])))

Premier = URIRef(DCSSO["GB1"])
g.add((Premier, RDF.type, DCSSO.League))
g.add((Premier, FOAF['name'], Literal("Premier League", datatype=XSD.string)))
g.add((Premier, DCSSO['hasCountry'], URIRef(DCSSO["GB"])))

Bundesliga = URIRef(DCSSO["L1"])
g.add((Bundesliga, RDF.type, DCSSO.League))
g.add((Bundesliga, FOAF['name'], Literal("Bundesliga", datatype=XSD.string)))
g.add((Bundesliga, DCSSO['hasCountry'], URIRef(DCSSO["DE"])))

We serialize the graph and save the output with turtle syntax.

In [ ]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'countries_leagues.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

We can now clean the graph.

In [ ]:
# Construct the soccer stats ontology namespace
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("dcsso", DCSSO)

#### Teams

We load the dictionary with team IDs, to understand which teams need to be stored in the DB.

In [273]:
%%time

with open('teamdict.pkl', 'rb') as f:
    teamIDDict = pickle.load(f)

for tID in set(teamIDDict.values()):
    team = teams.loc[tID]
    Team = URIRef(DCSSO["team"+str(tID)])
    g.add((Team, RDF.type, DCSSO.Team))
    g.add((Team, FOAF.name, Literal(cleanString(team['name']), datatype=XSD.string)))
    g.add((Team, DCSSO['participatesIn'], URIRef(DCSSO[team['domestic_competition_id']])))

CPU times: total: 31.2 ms
Wall time: 73.7 ms


In [274]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'teams.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 31.2 ms
Wall time: 44.7 ms


In [275]:
# Construct the soccer stats ontology namespace
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("dcsso", DCSSO)

#### Team participations

In [276]:
%%time

for tID in set(teamIDDict.values()):
    Team = URIRef(DCSSO["team"+str(tID)])
    for year in range(2014, 2020):
        #check if there are any games for this team in season starting in a year
        if((games[games['season'] == year]['home_club_id'] == tID).any() == True):
            #if there are any, 'year' represents a season in which this team has played in its domestic top league
            Participation = URIRef(DCSSO["part"+str(tID)+"s"+str(year)])
            g.add((Participation, RDF.type, DCSSO.SeasonalParticipation))
            g.add((Team, DCSSO['hasParticipation'], Participation))
            g.add((Participation, DCSSO['season'], Literal(year, datatype=XSD.int)))

In [277]:
%%time

# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'team_part.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 141 ms
Wall time: 207 ms


In [278]:
# Construct the soccer stats ontology namespace
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("dcsso", DCSSO)

#### Players

In [279]:
%%time

players['first_name'] = players['first_name'].fillna("")
players['last_name'] = players['last_name'].fillna("")
players['country_of_citizenship'] = players['country_of_citizenship'].fillna("")
players['height_in_cm'] = players['height_in_cm'].fillna(0)
players['date_of_birth'] = players['date_of_birth'].fillna("")

#collect all player IDs in the files produced above
plIDSet = set()
for n in range(14, 20):
    tmpstats = pd.read_csv('stats'+str(n)+str(n+1)+'_IDs.csv', sep=',')
    for ind, row in tmpstats.iterrows():
        if(row['trID'] != 0):
            plIDSet.add(row['trID'])

c = 0
for id in plIDSet:
    c+=1
    if(int(100*(c-1)/len(plIDSet)) < int(100*c/len(plIDSet))):
        clear_output(wait=True)
        print(str(int(100*c/len(plIDSet)))+"% completed."+"."*(c%3))
    player = players.loc[id]
    # the node has the namespace + the transfermarkt ID as URI
    ref = "player"+str(id)
    Footballer = URIRef(DCSSO[ref])
    g.add((Footballer, RDF.type, DCSSO.Footballer))
    if(player['first_name'] != ""):
        g.add((Footballer, FOAF['firstName'], Literal(cleanString(player['first_name']), datatype=XSD.string)))
    if(player['last_name'] != ""):
        g.add((Footballer, FOAF['familyName'], Literal(cleanString(player['last_name']), datatype=XSD.string)))
    if(player['date_of_birth'] != ""):
        g.add((Footballer, DCSSO['birthdate'], Literal(player['date_of_birth'][:4], datatype=XSD.int)))
    g.add((Footballer, FOAF['name'], Literal(cleanString(player['name']), datatype=XSD.string)))
    if(player['country_of_citizenship'] != ""):
        g.add((Footballer, DCSSO['hasCitizenship'], DCSSO[countries.loc[player['country_of_citizenship']]['Alpha-2 code']]))
    if(player['height_in_cm'] != 0 and np.isnan(player['height_in_cm']) == False):
        g.add((Footballer, DCSSO['height'], Literal(player['height_in_cm'], datatype=XSD.int)))

    if(player['position'] != "Missing"):
        subPosition = player['sub_position'].replace(" ", "").replace("-", "")
        if(player['position'] == "Goalkeeper" or player['position'] == "Defender"):
            g.add((Footballer, DCSSO['position'], DCSSO[player['position']]))
        elif(player['position'] == "Midfield"):
            g.add((Footballer, DCSSO['position'], DCSSO["Midfielder"]))
            subPosition += "er"
        else:
            g.add((Footballer, DCSSO['position'], DCSSO["Forward"]))
    
        g.add((Footballer, DCSSO['subPosition'], DCSSO[subPosition]))

100% completed.
CPU times: total: 3.64 s
Wall time: 7.04 s


In [280]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'players.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 1.61 s
Wall time: 2.92 s


In [335]:
# Construct the soccer stats ontology namespace
DCSSO = Namespace("http://www.dei.unipd.it/db2/dcsso#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("dcsso", DCSSO)

#### Statistics

Here, we finally store the statistics available for each player in a season.

In [336]:
%%time

y=2019
shortSeas = "1920"
stats = pd.read_csv('stats'+str(shortSeas)+'_IDs.csv', sep=',')

c=0
for index, row in stats.iterrows():
    c+=1
    if(int(100*(c-1)/len(stats)) < int(100*c/len(stats))):
        clear_output(wait=True)
        print(str(int(100*c/len(stats)))+"% completed."+"."*(c%3))
    tmId = str(row['trID'])
    if(tmId == "0"):
        continue
    # the node has the namespace + the transfermarkt ID as URI; we don't create the node for the footballer here
    ref = "player"+tmId
    Footballer = URIRef(DCSSO[ref])

    teamIDs = str(row['team0ID'])
    if(row['team1ID'] != 0):
        teamIDs += "_"+str(row['team1ID'])
        MembClassURI = DCSSO.SeasonalAggrMembership
    else:
        MembClassURI = DCSSO.SeasonalMembership
    Memb = URIRef(DCSSO["memb"+tmId+"s"+str(y)+"t"+teamIDs])
    g.add((Memb, RDF.type, MembClassURI))
    g.add((Footballer, DCSSO['hasMembership'], Memb))
    g.add((Memb, DCSSO['season'], Literal(str(y), datatype=XSD.int)))
    g.add((Memb, DCSSO['forTeam'], URIRef(DCSSO["team"+str(row['team0ID'])])))
    if(row['team1ID'] != 0):
        g.add((Memb, DCSSO['forTeam'], URIRef(DCSSO["team"+str(row['team1ID'])])))

    #statistics
    g.add((Memb, DCSSO['games'], Literal(row['games'], datatype=XSD.int)))
    g.add((Memb, DCSSO['minutes'], Literal(row['minutes_played'], datatype=XSD.int)))
    g.add((Memb, DCSSO['goals'], Literal(row['goals'], datatype=XSD.int)))
    g.add((Memb, DCSSO['npg'], Literal(row['npg'], datatype=XSD.int)))
    g.add((Memb, DCSSO['assists'], Literal(row['assists'], datatype=XSD.int)))
    g.add((Memb, DCSSO['shots'], Literal(row['shots'], datatype=XSD.int)))
    g.add((Memb, DCSSO['yellowCards'], Literal(row['yellow_cards'], datatype=XSD.int)))
    g.add((Memb, DCSSO['redCards'], Literal(row['red_cards'], datatype=XSD.int)))
    
    g.add((Memb, DCSSO['xG'], Literal(row['xG'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xA'], Literal(row['xA'], datatype=XSD.double)))
    g.add((Memb, DCSSO['npxG'], Literal(row['npxG'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xG90'], Literal(row['xG90'], datatype=XSD.double)))
    g.add((Memb, DCSSO['xA90'], Literal(row['xA90'], datatype=XSD.double)))
    g.add((Memb, DCSSO['npxG90'], Literal(row['npxG90'], datatype=XSD.double)))

    #key pass. set to -1 implies that we miss the following data
    if(row['key_passes'] != -1):
        g.add((Memb, DCSSO['keyPasses'], Literal(row['key_passes'], datatype=XSD.int)))
        g.add((Memb, DCSSO['xGBuildup'], Literal(row['xGBuildup'], datatype=XSD.double)))
        g.add((Memb, DCSSO['xGChain'], Literal(row['xGChain'], datatype=XSD.double)))

100% completed..
CPU times: total: 1.64 s
Wall time: 4.45 s


In [337]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'stats'+shortSeas+'.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

--- saving serialization ---
CPU times: total: 1.81 s
Wall time: 4.26 s


In [ ]:
# !pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """SELECT ?oLabel
WHERE
{
wd:Q192923 skos:altLabel ?o.
FILTER(isLiteral(?o))
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en" }
}"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result["oLabel"]["value"])